In [1]:
import numpy
from helper import l2_norm, poisson_2d_jacobi, poisson_solution

In [2]:
nx = 101
ny = 101
ymin, ymax = -0.5, 0.5
xmin, xmax = 0.0, 1.0
Lx = xmax - xmin
Ly = ymax - ymin
dx = Lx/(nx - 1)
dy = Ly/(ny - 1)

x = numpy.linspace(xmin, xmax, num=nx)
y = numpy.linspace(ymin, ymax, num=ny)
X, Y = numpy.meshgrid(x, y)

# create Poisson source term
b = (-2.0 * (numpy.pi / 2)**2 *
    numpy.sin(numpy.pi * X / Lx) *
    numpy.cos(numpy.pi * Y / Ly))

# set initial guess
p0 = numpy.zeros((ny, nx))

# compute exact solution of Poisson system
pe = poisson_solution(x, y, Lx, Ly)

In [3]:
def poisson_2d_steepest_descent(p0, b, dx, dy, maxiter=20000, rtol=1e-6):
    def A(p):
        #compute laplacian of p
        return((p[1:-1, :-2] - 2 * p[1:-1, 1:-1] + p[1:-1, 2:]) / dx**2 +
               (p[:-2, 1:-1] - 2 * p[1:-1, 1:-1] + p[2:, 1:-1]) / dy**2)
    p = p0.copy()
    diff = rtol + 1
    ite = 0
    rk = numpy.zeros_like(p)
    Ar = numpy.zeros_like(p)
    while diff > rtol and ite < maxiter:
        pk = p.copy()
        # compute residual rk
        rk[1:-1, 1:-1] = b[1:-1, 1:-1] - A(pk)
        # compute step size alpha
        Ar[1:-1, 1:-1] = A(rk)
        alpha = numpy.sum(rk * rk) / numpy.sum(rk * Ar)
        # update solution
        p = pk + alpha * rk
        # compute relative difference
        diff = l2_norm(p, pk)
        # increment iteration index
        ite += 1
    return p, ite, diff

In [4]:
p, ites, diff = poisson_2d_steepest_descent(p0, b, dx, dy, rtol=1e-6)
ites, diff

(2, 1.2374874917997386e-16)

In [5]:
b = (numpy.sin(numpy.pi * X / Lx) *
     numpy.cos(numpy.pi * Y / Ly) +
     numpy.sin(6.0 * numpy.pi * X / Lx) *
     numpy.sin(6.0 * numpy.pi * Y / Ly))

In [10]:
def poisson_2d_conjugate_gradient(p0, b, dx, dy, maxiter=20000, rtol=1e-6):
    def A(p):
        return((p[1:-1, :-2] - 2 * p[1:-1, 1:-1] + p[1:-1, 2:]) / dx**2 +
               (p[:-2, 1:-1] - 2 * p[1:-1, 1:-1] + p[2:, 1:-1]) / dy**2)
    p = p0.copy()
    r = numpy.zeros_like(p)
    Ad = numpy.zeros_like(p)

    diff = rtol + 1
    ite = 0
    r[1:-1, 1:-1] = b[1:-1, 1:-1] - A(p)
    d = r.copy()
    while diff > rtol and ite < maxiter:
        rk = r.copy()
        pk = p.copy()
        Ad[1:-1, 1:-1] = A(d)
        alpha = numpy.sum(r * r) / numpy.sum(d * Ad)
        
        p = pk + alpha * d
        r = rk - alpha * Ad
        beta = numpy.sum(r * r) / numpy.sum(rk * rk)
        d = r + beta * d
        diff = l2_norm(p, pk)

        ite += 1
    return p, ite, diff
        

In [11]:
p, ites, diff = poisson_2d_conjugate_gradient(p0, b, dx, dy, rtol=1e-10)
print(ites, diff)

3 2.4646085821771386e-14
